# 学習データ収集
# Flickr 上の著作権フリー画像をダウンロードする

flickr.photos.search　というメソッドを利用する。くわしくは、

PHP での説明であるが、[flickr APIの使い方まとめ(サンプルコード付き)](https://syncer.jp/flickr-api-matome) が参考になる。

### Anaconda 標準でない必要ライブラリ
- tensorflow
- keras
- flickrapi

tensorflow は conda で導入可能。ほか２つは pip install で導入。

### Flickr API の取得
「Flickr API key 取得」といった語で検索したらやり方を解説したサイトが出てくる。説明省略

In [ ]:
from flickrapi import FlickrAPI
from urllib.request import urlretrieve
from pprint import pprint
import os, time, sys
from PIL import Image

key = '581aad48f15c7be10c83abb9c99ba888'
secret = '881ad19fd75c5c3a'
flickr = FlickrAPI(key, secret, format= 'parsed-json')

# ライセンスについて Flickrの写真には、次の分類に従った著作権ラベルが付けられている

- 0 All Rights Reserved
- 1 Attribution-NonCommercial-ShareAlike License
- 2 Attribution-NonCommercial License
- 3 Attribution-NonCommercial-NoDerivs License
- 4 Attribution License
- 5 Attribution-ShareAlike License
- 6 Attribution-NoDerivs License
- 7 No known copyright restrictions
- 8 United States Government Work
- 9 Public Domain Dedication (CC0)
- 10 Public Domain Mark

In [ ]:
for li in flickr.photos.licenses.getInfo()['licenses']['license']:
    print(li['id'],li['name'])

# キーワードによる写真の収集

In [ ]:
search_for = "Zelkova leaves" # ケヤキの葉
search_for = "osmanthus" # 金木犀
search_for = "dogwood leaf" # ハナミズキ
search_for = "Leaf Magnolia" # ハモクレン
search_for = "sieboldiana" # ハナノキ
# search_for = "yoshino leaf cherry" # ソメイヨシノ  と ヤマザクラは Google 検索で

savedir = "./"  + search_for
if not os.path.exists(savedir):
    os.mkdir(savedir)

def  getPictures(page=1):
    res = flickr.photos.search(
        text = search_for,  # 検索キーワード
        page = page,
        per_page = 100,  # 検索件数
        media = 'photos',  # 画像のみを検索対象とする
        sort = 'relevance', # 関連の強い順に並べ替え
        safe_search = 1, # 不適切な画像は除く
        extras =  'url_s,license') # 取得するデータ項目
    return res

'''
url_sq … 75x75サイズの画像ファイルURL。
url_t … 最大100pxの画像ファイルURL。
url_s … 最大240pxの画像ファイルURL。
url_q … 150x150の画像ファイルURL。
url_m … 最大500pxの画像ファイルURL。
url_n … 最大320pxの画像ファイルURL。
url_z … 最大640pxの画像ファイルURL。
url_c … 最大800pxの画像ファイルURL。
url_l … 最大1024pxの画像ファイルURL。
url_o … オリジナルサイズの画像ファイルURL。
'''

sleep_time = 1
page = 0
pic_count = 0
pic_needed = 200

while True :
    photos =  getPictures(page = page)['photos']
    # pprint(photos)
    print("page =",page, "pic_count=",pic_count)
    try:
        for i, photo in enumerate(photos['photo']):
            lic = photo['license']
            if lic != '0' : # 完全著作権保護されているものはのぞく
                url = photo['url_s']
                filepath = savedir + '/' +  photo['id'] + '.jpg'
                if os.path.exists(filepath): continue
                # print(str(i+1) + ':url=', url_sq, 'license', lic)
                urlretrieve(url, filepath)
                img = Image.open(filepath, 'r')
                time.sleep(sleep_time)
                os.remove(filepath) # 一旦削除
                if img.width >= 160  and img.height >= 160 :  # 短辺が160以上の画像のみ採用
                    pic_count += 1
                    x1= int((img.width - 160)/2)
                    x2 = x1+160
                    y1= int((img.height - 160)/2)
                    y2 = y1+160
                    img = img.crop((x1,y1,x2,y2)).resize((128,128))  #128x128にリサイズ
                    img.save(filepath)
                if pic_count == pic_needed:break
    except:
        import traceback
        traceback.print_exc()
    page +=1
    if pic_count >= pic_needed: break